In [315]:
import math
import numpy as np
import pandas as pd
# import scipy.stats as stats
import statsmodels.api as sm

import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import seaborn as sns
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)

from scipy import stats
from sklearn.linear_model import LogisticRegression as lr
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics
# Enabled to remove warnings for demo purposes.
import warnings
warnings.filterwarnings('ignore')

In [316]:
df = pd.read_csv('./healthcare-dataset-stroke-data.csv')

In [317]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [318]:
df_na = df.dropna()
df_na.reset_index(inplace=True)

In [319]:
pd.isnull(df_na).any()

index                False
id                   False
gender               False
age                  False
hypertension         False
heart_disease        False
ever_married         False
work_type            False
Residence_type       False
avg_glucose_level    False
bmi                  False
smoking_status       False
stroke               False
dtype: bool

In [320]:
df_na

,index,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,5104,14180,Female,13.0,0,0,No,children,Rural,103.08,18.6,Unknown,0
4905,5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
4906,5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
4907,5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [321]:
df_na['ever_married'] = df_na.ever_married.map({'Yes':1, 'No':0})
# For gender, 1 means Male, 0 means Female
df_na['gender'] = df_na.gender.map({'Male':1, 'Female':0})
# For Residence_type, 1 means Urban, 0 means Rural
df_na['Residence_type'] = df_na.Residence_type.map({'Urban':1, 'Rural':0})

In [322]:
pd.isnull(df_na).any()
df_na = df_na.dropna()
df_na.reset_index(drop=True,inplace=True)
pd.isnull(df_na).any()

index                False
id                   False
gender               False
age                  False
hypertension         False
heart_disease        False
ever_married         False
work_type            False
Residence_type       False
avg_glucose_level    False
bmi                  False
smoking_status       False
stroke               False
dtype: bool

In [323]:
df_na

,index,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,9046,1.0,67.0,0,1,1,Private,1,228.69,36.6,formerly smoked,1
1,2,31112,1.0,80.0,0,1,1,Private,0,105.92,32.5,never smoked,1
2,3,60182,0.0,49.0,0,0,1,Private,1,171.23,34.4,smokes,1
3,4,1665,0.0,79.0,1,0,1,Self-employed,0,174.12,24.0,never smoked,1
4,5,56669,1.0,81.0,0,0,1,Private,1,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,5104,14180,0.0,13.0,0,0,0,children,0,103.08,18.6,Unknown,0
4904,5106,44873,0.0,81.0,0,0,1,Self-employed,1,125.20,40.0,never smoked,0
4905,5107,19723,0.0,35.0,0,0,1,Self-employed,0,82.99,30.6,never smoked,0
4906,5108,37544,1.0,51.0,0,0,1,Private,0,166.29,25.6,formerly smoked,0


In [324]:
def Glucose_check(x):
    if x >= 140:
        return 1
    else:
        return 0
# 1 means it is higher than the bound. 0 means lower.
df_na['treatment_glucose_level'] = df_na.avg_glucose_level.apply(Glucose_check)

In [325]:
df_data = df_na.drop(columns=['index','id','avg_glucose_level'])
df_data

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,bmi,smoking_status,stroke,treatment_glucose_level
0,1.0,67.0,0,1,1,Private,1,36.6,formerly smoked,1,1
1,1.0,80.0,0,1,1,Private,0,32.5,never smoked,1,0
2,0.0,49.0,0,0,1,Private,1,34.4,smokes,1,1
3,0.0,79.0,1,0,1,Self-employed,0,24.0,never smoked,1,1
4,1.0,81.0,0,0,1,Private,1,29.0,formerly smoked,1,1
...,...,...,...,...,...,...,...,...,...,...,...
4903,0.0,13.0,0,0,0,children,0,18.6,Unknown,0,0
4904,0.0,81.0,0,0,1,Self-employed,1,40.0,never smoked,0,0
4905,0.0,35.0,0,0,1,Self-employed,0,30.6,never smoked,0,0
4906,1.0,51.0,0,0,1,Private,0,25.6,formerly smoked,0,1


In [326]:
df_encoded = pd.get_dummies(df_data, columns = ['work_type','smoking_status'], \
                           prefix = {'work_type':'work_type', 'smoking_status' : 'smoking_status'}, drop_first=False)
df_encoded

,gender,age,hypertension,heart_disease,ever_married,Residence_type,bmi,stroke,treatment_glucose_level,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1.0,67.0,0,1,1,1,36.6,1,1,0,0,1,0,0,0,1,0,0
1,1.0,80.0,0,1,1,0,32.5,1,0,0,0,1,0,0,0,0,1,0
2,0.0,49.0,0,0,1,1,34.4,1,1,0,0,1,0,0,0,0,0,1
3,0.0,79.0,1,0,1,0,24.0,1,1,0,0,0,1,0,0,0,1,0
4,1.0,81.0,0,0,1,1,29.0,1,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,0.0,13.0,0,0,0,0,18.6,0,0,0,0,0,0,1,1,0,0,0
4904,0.0,81.0,0,0,1,1,40.0,0,0,0,0,0,1,0,0,0,1,0
4905,0.0,35.0,0,0,1,0,30.6,0,0,0,0,0,1,0,0,0,1,0
4906,1.0,51.0,0,0,1,0,25.6,0,1,0,0,1,0,0,0,1,0,0


In [327]:
df_treatment = df_encoded[df_encoded['treatment_glucose_level']==1]
df_control = df_encoded[df_encoded['treatment_glucose_level']==0]
treat_mean = df_treatment.mean()
treat_std = df_treatment.std()
control_mean = df_control.mean()
control_std = df_control.std()

In [328]:
X_list = ['gender','age','hypertension','heart_disease','ever_married','Residence_type','bmi','work_type_Govt_job',
          'work_type_Never_worked','work_type_Private','work_type_Self-employed','work_type_children','smoking_status_Unknown',
          'smoking_status_formerly smoked','smoking_status_never smoked','smoking_status_smokes']
X_descrip = ['0 to Female and 1 to Male','age of the sample','observation of hypertension','observation of heart_disease',
             'Marriage states','0 to Rural and 1 to Urban','bmi','Whether is government job',
             'Whether is never worked','Whether job is private','Whether job is self-employed','Whether job is children',
             'Whether smoking status is unknown','Whether is formerly smoked','Whether is never smoked','Whether is smokes']

In [329]:
summary_list = []

for i in range(0, len(X_list)):
    summary_list.append([X_list[i],X_descrip[i],control_mean[X_list[i]],control_std[X_list[i]],treat_mean[X_list[i]],treat_std[X_list[i]],
                    stats.ttest_ind(df_control[X_list[i]], df_treatment[X_list[i]], equal_var=False)[0]])

summary_df = pd.DataFrame(summary_list,columns=
                          ['Label', 'Variable Description','Controls Mean','Controls STD','Treated Mean','Treated STD','t-Statistics'])
summary_df

,Label,Variable Description,Controls Mean,Controls STD,Treated Mean,Treated STD,t-Statistics
0,gender,0 to Female and 1 to Male,0.398411,0.489630,0.472149,0.499555,-3.740135
1,age,age of the sample,40.536081,22.300920,55.720477,19.421345,-19.284613
2,hypertension,observation of hypertension,0.069090,0.253638,0.217507,0.412824,-9.550201
3,heart_disease,observation of heart_disease,0.035388,0.184780,0.127321,0.333553,-7.365844
4,ever_married,Marriage states,0.620366,0.485354,0.831565,0.374501,-13.556302
5,Residence_type,0 to Rural and 1 to Urban,0.506259,0.500021,0.513263,0.500156,-0.353752
6,bmi,bmi,28.298387,7.679817,32.179045,7.997494,-12.331939
7,work_type_Govt_job,Whether is government job,0.125662,0.331508,0.143236,0.350546,-1.276873
8,work_type_Never_worked,Whether is never worked,0.004815,0.069229,0.002653,0.051468,1.000831
9,work_type_Private,Whether job is private,0.570053,0.495128,0.586207,0.492839,-0.827426


In [330]:
df = df_encoded

In [331]:
Y = df.stroke

df_data = df.loc[:,df.columns !='stroke']
T = df_data.treatment_glucose_level
X = df_data.loc[:,df_data.columns !='treatment_glucose_level']

In [332]:
X_g = ['age','hypertension','heart_disease','bmi']

In [333]:
T.mean()

0.15362673186634065

In [334]:
model_initial = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:3.75}))])
model_initial.fit(X.loc[:,X_g], T)

predictions_binary = model_initial.predict(X.loc[:,X_g])
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

Accuracy: 0.7763

Confusion matrix:
[[3421  733]
 [ 365  389]]

F1 score is: 0.4147


In [335]:
likelihood_ratio_table = []
X_l = X_g.copy()

while not likelihood_ratio_table or max([x for x in likelihood_ratio_table[-1].values() if x != '-']) >= 1:
    if likelihood_ratio_table:
        max_likelihood_ratio = max([x for x in likelihood_ratio_table[-1].values() if x != '-'])
        next_label = [key for key in likelihood_ratio_table[-1] if likelihood_ratio_table[-1][key] == max_likelihood_ratio][0]
        X_l.append(next_label)
    model = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:3.75}))])
    model.fit(X.loc[:,X_l], T)
    step = {}
    for x in X.columns:
        if x in X_l:
            step[x] = '-'
        else:
            X_new = X_l.copy()
            X_new.append(x)
            model_new = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:3.75}))])
            model_new.fit(X.loc[:,X_new], T)
            
            step[x] = -2*((-metrics.log_loss(T, model.predict_proba(X.loc[:,X_l]))*len(T))
                      -(-metrics.log_loss(T, model_new.predict_proba(X.loc[:,X_new]))*len(T)))
    likelihood_ratio_table.append(step)

In [336]:
X_l

['age',
 'hypertension',
 'heart_disease',
 'bmi',
 'gender',
 'work_type_children',
 'smoking_status_Unknown',
 'ever_married',
 'work_type_Never_worked']

In [337]:
linear_terms_list = []
for x in X.columns:
    row = [step[x] for step in likelihood_ratio_table]
    row.insert(0, x)
    linear_terms_list.append(row)
    
columns = ['Label']
for i in range(0,len(likelihood_ratio_table)):
    step = 'Step ' + str(i+1)
    columns.append(step)
    
linear_terms_df = pd.DataFrame(linear_terms_list,columns = columns)
linear_terms_df

,Label,Step 1,Step 2,Step 3,Step 4,Step 5,Step 6
0,gender,18.388731,-,-,-,-,-
1,age,-,-,-,-,-,-
2,hypertension,-,-,-,-,-,-
3,heart_disease,-,-,-,-,-,-
4,ever_married,0.644154,0.538906,2.441166,2.248674,-,-
5,Residence_type,0.076674,0.08478,0.078481,0.091419,0.096145,0.078832
6,bmi,-,-,-,-,-,-
7,work_type_Govt_job,0.121511,0.096339,-0.012854,-0.003263,0.010848,0.005688
8,work_type_Never_worked,0.637735,0.571155,0.95948,0.987351,1.281822,-
9,work_type_Private,-0.023031,0.076018,0.239776,0.301466,0.264646,0.396416


In [338]:
model_linear = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:3.75}))])
model_linear.fit(X.loc[:,X_l], T)

predictions_binary = model_linear.predict(X.loc[:,X_l])
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

Accuracy: 0.7692

Confusion matrix:
[[3360  794]
 [ 339  415]]

F1 score is: 0.4228


In [339]:
X_copy = X.loc[:,X_l]

for i in range(0, len(X_l)):
    j = i
    while j < len(X_l):
        X_copy[X_l[i]+'*'+X_l[j]] = X_copy[X_l[i]] * X_copy[X_l[j]]
        j += 1

X_copy

,age,hypertension,heart_disease,bmi,gender,work_type_children,smoking_status_Unknown,ever_married,work_type_Never_worked,age*age,...,work_type_children*work_type_children,work_type_children*smoking_status_Unknown,work_type_children*ever_married,work_type_children*work_type_Never_worked,smoking_status_Unknown*smoking_status_Unknown,smoking_status_Unknown*ever_married,smoking_status_Unknown*work_type_Never_worked,ever_married*ever_married,ever_married*work_type_Never_worked,work_type_Never_worked*work_type_Never_worked
0,67.0,0,1,36.6,1.0,0,0,1,0,4489.0,...,0,0,0,0,0,0,0,1,0,0
1,80.0,0,1,32.5,1.0,0,0,1,0,6400.0,...,0,0,0,0,0,0,0,1,0,0
2,49.0,0,0,34.4,0.0,0,0,1,0,2401.0,...,0,0,0,0,0,0,0,1,0,0
3,79.0,1,0,24.0,0.0,0,0,1,0,6241.0,...,0,0,0,0,0,0,0,1,0,0
4,81.0,0,0,29.0,1.0,0,0,1,0,6561.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,13.0,0,0,18.6,0.0,1,1,0,0,169.0,...,1,1,0,0,1,0,0,0,0,0
4904,81.0,0,0,40.0,0.0,0,0,1,0,6561.0,...,0,0,0,0,0,0,0,1,0,0
4905,35.0,0,0,30.6,0.0,0,0,1,0,1225.0,...,0,0,0,0,0,0,0,1,0,0
4906,51.0,0,0,25.6,1.0,0,0,1,0,2601.0,...,0,0,0,0,0,0,0,1,0,0


In [354]:
likelihood_ratio_table = []
X_2 = X_l.copy()

while not likelihood_ratio_table or max([x for x in likelihood_ratio_table[-1].values() if x != '-']) >= 2.71:
    if likelihood_ratio_table:
        max_likelihood_ratio = max([x for x in likelihood_ratio_table[-1].values() if x != '-'])
        next_label = [key for key in likelihood_ratio_table[-1] if likelihood_ratio_table[-1][key] == max_likelihood_ratio][0]
        X_2.append(next_label)
    model = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:3.75}))])
    model.fit(X_copy.loc[:,X_2], T)
    step = {}
    for x in X_copy.columns:
        if x in X_2:
            step[x] = '-'
        else:
            X_new = X_2.copy()
            X_new.append(x)
            model_new = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:3.75}))])
            model_new.fit(X_copy.loc[:,X_new], T)
            
            step[x] = -2*((-metrics.log_loss(T, model.predict_proba(X_copy.loc[:,X_2]))*len(T))
                      -(-metrics.log_loss(T, model_new.predict_proba(X_copy.loc[:,X_new]))*len(T)))
    likelihood_ratio_table.append(step)

In [355]:
final_terms_list = []
for x in X_copy.columns:
    row = [step[x] for step in likelihood_ratio_table]
    row.insert(0, x)
    final_terms_list.append(row)
    
final_columns = ['Label']
for i in range(0,len(likelihood_ratio_table)):
    step = 'Step ' + str(i+1)
    final_columns.append(step)
    
final_terms_df = pd.DataFrame(final_terms_list,columns = final_columns)
final_terms_df

,Label,Step 1,Step 2,Step 3,Step 4,Step 5,Step 6,Step 7,Step 8,Step 9
0,age,-,-,-,-,-,-,-,-,-
1,hypertension,-,-,-,-,-,-,-,-,-
2,heart_disease,-,-,-,-,-,-,-,-,-
3,bmi,-,-,-,-,-,-,-,-,-
4,gender,-,-,-,-,-,-,-,-,-
5,work_type_children,-,-,-,-,-,-,-,-,-
6,smoking_status_Unknown,-,-,-,-,-,-,-,-,-
7,ever_married,-,-,-,-,-,-,-,-,-
8,work_type_Never_worked,-,-,-,-,-,-,-,-,-
9,age*age,-0.395083,0.242698,0.222412,0.634892,0.794139,0.192182,0.161724,0.239706,-0.05791


In [356]:
X_2

['age',
 'hypertension',
 'heart_disease',
 'bmi',
 'gender',
 'work_type_children',
 'smoking_status_Unknown',
 'ever_married',
 'work_type_Never_worked',
 'age*bmi',
 'heart_disease*gender',
 'work_type_children*smoking_status_Unknown',
 'hypertension*bmi',
 'age*smoking_status_Unknown',
 'hypertension*smoking_status_Unknown',
 'age*work_type_Never_worked',
 'age*ever_married']

In [357]:
model_final = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:3.75}))])
model_final.fit(X_copy.loc[:,X_2], T)

predictions_binary = model_final.predict(X_copy.loc[:,X_2])
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

Accuracy: 0.7802

Confusion matrix:
[[3405  749]
 [ 330  424]]

F1 score is: 0.4401


In [358]:
treat_index = []
control_index = []
for i in range(0,len(T)):
    if T[i] == 1:
        treat_index.append(i)
    elif T[i] == 0:
        control_index.append(i)

In [359]:
predictions = model_final.predict_proba(X_copy.loc[:,X_2])

ate_ipw = sum([T[i]*Y[i]/predictions[i][1] for i in treat_index]) / sum([T[i]/predictions[i][1] for i in treat_index]) 
- sum([(1-T[i])*Y[i]/(1-predictions[i][1]) for i in control_index]) / sum([(1-T[i])/(1-predictions[i][1]) for i in control_index])

-0.04405140939553362